In [2]:
import gzip
import pickle
import numpy as np

def load_data():
    f = gzip.open('/content/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [3]:
import random
import numpy as np

class Network:

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {}: {} / {}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].T)
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].T, delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].T)
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))


In [7]:
training_data, validation_data, test_data = load_data_wrapper()

# Basic network
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)




Epoch 0: 9077 / 10000
Epoch 1: 9199 / 10000
Epoch 2: 9314 / 10000
Epoch 3: 9319 / 10000
Epoch 4: 9329 / 10000
Epoch 5: 9373 / 10000
Epoch 6: 9377 / 10000
Epoch 7: 9398 / 10000
Epoch 8: 9396 / 10000
Epoch 9: 9409 / 10000
Epoch 10: 9446 / 10000
Epoch 11: 9445 / 10000
Epoch 12: 9459 / 10000
Epoch 13: 9466 / 10000
Epoch 14: 9421 / 10000
Epoch 15: 9476 / 10000
Epoch 16: 9492 / 10000
Epoch 17: 9484 / 10000
Epoch 18: 9492 / 10000
Epoch 19: 9472 / 10000
Epoch 20: 9473 / 10000
Epoch 21: 9487 / 10000
Epoch 22: 9470 / 10000
Epoch 23: 9495 / 10000
Epoch 24: 9502 / 10000
Epoch 25: 9487 / 10000
Epoch 26: 9507 / 10000
Epoch 27: 9501 / 10000
Epoch 28: 9497 / 10000
Epoch 29: 9505 / 10000


In [8]:
# Larger network
net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)



Epoch 0: 5693 / 10000
Epoch 1: 5807 / 10000
Epoch 2: 5834 / 10000
Epoch 3: 5836 / 10000
Epoch 4: 5877 / 10000
Epoch 5: 5875 / 10000
Epoch 6: 5889 / 10000
Epoch 7: 6062 / 10000
Epoch 8: 6808 / 10000
Epoch 9: 7626 / 10000
Epoch 10: 7679 / 10000
Epoch 11: 7695 / 10000
Epoch 12: 7681 / 10000
Epoch 13: 7684 / 10000
Epoch 14: 7741 / 10000
Epoch 15: 7744 / 10000
Epoch 16: 7813 / 10000
Epoch 17: 8622 / 10000
Epoch 18: 8635 / 10000
Epoch 19: 8651 / 10000
Epoch 20: 8652 / 10000
Epoch 21: 8647 / 10000
Epoch 22: 8669 / 10000
Epoch 23: 8675 / 10000
Epoch 24: 8690 / 10000
Epoch 25: 8676 / 10000
Epoch 26: 8697 / 10000
Epoch 27: 8659 / 10000
Epoch 28: 8692 / 10000
Epoch 29: 8696 / 10000


In [9]:
# High learning rate
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 100.0, test_data=test_data)

Epoch 0: 1039 / 10000
Epoch 1: 1039 / 10000
Epoch 2: 1039 / 10000
Epoch 3: 1039 / 10000
Epoch 4: 1039 / 10000
Epoch 5: 1039 / 10000
Epoch 6: 1039 / 10000
Epoch 7: 1039 / 10000
Epoch 8: 1039 / 10000
Epoch 9: 1039 / 10000
Epoch 10: 1039 / 10000
Epoch 11: 1039 / 10000
Epoch 12: 1039 / 10000
Epoch 13: 1039 / 10000
Epoch 14: 1039 / 10000
Epoch 15: 1039 / 10000
Epoch 16: 1039 / 10000
Epoch 17: 1039 / 10000
Epoch 18: 1039 / 10000
Epoch 19: 1039 / 10000
Epoch 20: 1039 / 10000
Epoch 21: 1039 / 10000
Epoch 22: 1039 / 10000
Epoch 23: 1039 / 10000
Epoch 24: 1039 / 10000
Epoch 25: 1039 / 10000
Epoch 26: 1039 / 10000
Epoch 27: 1039 / 10000
Epoch 28: 1039 / 10000
Epoch 29: 1039 / 10000


In [11]:
# Low learning rate
net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 0.001, test_data=test_data)

Epoch 0: 989 / 10000
Epoch 1: 1033 / 10000
Epoch 2: 1532 / 10000
Epoch 3: 1641 / 10000
Epoch 4: 1702 / 10000
Epoch 5: 1741 / 10000
Epoch 6: 1777 / 10000
Epoch 7: 1814 / 10000
Epoch 8: 1841 / 10000
Epoch 9: 1883 / 10000
Epoch 10: 1912 / 10000
Epoch 11: 1956 / 10000
Epoch 12: 2011 / 10000
Epoch 13: 2048 / 10000
Epoch 14: 2079 / 10000
Epoch 15: 2110 / 10000
Epoch 16: 2143 / 10000
Epoch 17: 2209 / 10000
Epoch 18: 2251 / 10000
Epoch 19: 2297 / 10000
Epoch 20: 2355 / 10000
Epoch 21: 2406 / 10000
Epoch 22: 2450 / 10000
Epoch 23: 2496 / 10000
Epoch 24: 2538 / 10000
Epoch 25: 2578 / 10000
Epoch 26: 2623 / 10000
Epoch 27: 2655 / 10000
Epoch 28: 2700 / 10000
Epoch 29: 2732 / 10000
